# Építés a Meta család modelljeivel

## Bevezetés

Ebben a leckében szó lesz:

- A Meta család két fő modelljének, a Llama 3.1-nek és a Llama 3.2-nek a bemutatása
- Annak megértése, hogy melyik modellt milyen felhasználási esetekhez és helyzetekhez érdemes választani
- Kódrészlet, amely bemutatja az egyes modellek egyedi jellemzőit

## A Meta család modelljei

Ebben a leckében a Meta család, vagyis a "Llama csorda" két modelljét vizsgáljuk meg: a Llama 3.1-et és a Llama 3.2-t.

Ezek a modellek többféle változatban érhetők el, és megtalálhatók a Github Model piactéren. További információkat itt találsz arról, hogyan lehet Github Modellekkel [AI modelleket prototípusozni](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

Modellváltozatok:
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*Megjegyzés: A Llama 3 is elérhető a Github Modellek között, de ebben a leckében nem térünk ki rá*


## Llama 3.1

A 405 milliárd paraméterrel rendelkező Llama 3.1 az open source LLM kategóriába tartozik.

Ez a modell a korábbi Llama 3 továbbfejlesztett változata, amely a következőket kínálja:

- Nagyobb kontextusablak – 128k token az eddigi 8k helyett
- Nagyobb maximális kimeneti token – 4096 az eddigi 2048 helyett
- Jobb többnyelvű támogatás – a tanító tokenek számának növekedése miatt

Ezek lehetővé teszik, hogy a Llama 3.1 összetettebb felhasználási eseteket is kezeljen GenAI alkalmazások fejlesztésekor, például:
- Natív függvényhívás – lehetőség külső eszközök és függvények meghívására az LLM munkafolyamatán kívül
- Jobb RAG teljesítmény – a nagyobb kontextusablaknak köszönhetően
- Szintetikus adatok generálása – hatékony adatok létrehozása olyan feladatokhoz, mint például a finomhangolás


### Natív függvényhívás

A Llama 3.1-et finomhangolták, hogy hatékonyabban tudjon függvényeket vagy eszközöket meghívni. Két beépített eszközzel is rendelkezik, amelyeket a modell képes felismerni, ha a felhasználó promptja alapján szükség van rájuk. Ezek az eszközök:

- **Brave Search** – Friss információk, például időjárás lekérdezésére használható webes kereséssel
- **Wolfram Alpha** – Összetettebb matematikai számításokhoz használható, így nem kell saját függvényeket írni

Saját egyedi eszközöket is létrehozhatsz, amelyeket az LLM meghívhat.

Az alábbi kódrészletben:

- Meghatározzuk az elérhető eszközöket (brave_search, wolfram_alpha) a rendszer promptban.
- Egy felhasználói promptot küldünk, amely egy adott város időjárásáról érdeklődik.
- Az LLM válaszként egy eszközhívást fog küldeni a Brave Search eszköznek, ami így fog kinézni: `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Megjegyzés: Ez a példa csak az eszközhívást mutatja be, ha az eredményeket is szeretnéd megkapni, akkor ingyenes fiókot kell létrehoznod a Brave API oldalán, és magát a függvényt is definiálnod kell.*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Annak ellenére, hogy Llama 3.1 egy LLM, van egy korlátja: a multimodalitás. Ez azt jelenti, hogy nem képes különböző típusú bemeneteket, például képeket használni promptként, és ezekre válaszolni. Ez a képesség az egyik fő újdonsága a Llama 3.2-nek. Ezek a funkciók a következőket is tartalmazzák:

- Multimodalitás – képes szöveges és képes promptokat is értékelni
- Kis és közepes méretű változatok (11B és 90B) – ez rugalmas telepítési lehetőségeket biztosít,
- Csak szöveges változatok (1B és 3B) – ez lehetővé teszi, hogy a modellt edge / mobil eszközökön is futtassuk, alacsony késleltetéssel

A multimodális támogatás nagy előrelépést jelent a nyílt forráskódú modellek világában. Az alábbi kódrészlet egy képet és egy szöveges promptot is felhasznál, hogy elemzést kapjunk a képről a Llama 3.2 90B-től.

### Multimodális támogatás a Llama 3.2-vel


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## A tanulás itt nem ér véget, folytasd az utat

Miután befejezted ezt a leckét, nézd meg a [Generatív MI tanulási gyűjteményünket](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst), hogy tovább fejleszd a generatív MI-vel kapcsolatos tudásodat!



---

**Jogi nyilatkozat**:  
Ez a dokumentum AI fordítási szolgáltatás, a [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével készült. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti, eredeti nyelvű dokumentum tekintendő hiteles forrásnak. Kritikus információk esetén javasoljuk a professzionális, emberi fordítást. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
